In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column as fc
from tensorflow.keras.datasets import boston_housing

In [2]:
# Load the Dataset
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

57344/57026 [==============================] - 0s 0us/step


In [5]:
# Prepare Data Frames for easier processing
features = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

x_train_df = pd.DataFrame(x_train, columns=features)
x_test_df = pd.DataFrame(x_test, columns=features)
y_train_df = pd.DataFrame(y_train, columns=['MEDV'])
y_test_df = pd.DataFrame(y_test, columns=['MEDV'])
x_train_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65


In [6]:
# Gather all the features
feature_columns = []
for feature_name in features:
    feature_columns.append(fc.numeric_column(feature_name, dtype=tf.float32))

In [7]:
# Create input functions
def estimator_input_fun(df_data, df_label, epochs=10, shuffle=True, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(df_data), df_label))
        if shuffle:
            ds = ds.shuffle(100)
        ds = ds.batch(batch_size).repeat(epochs)
        return ds
    return input_function

train_input_fun = estimator_input_fun(x_train_df, y_train_df)
val_input_fun = estimator_input_fun(x_test_df, y_test_df, epochs=1, shuffle=False)


In [11]:
# Build & train the estimator
linear_est = tf.estimator.LinearRegressor(feature_columns=feature_columns, model_dir='logs/func/')
linear_est.train(train_input_fun, steps=100)
result = linear_est.evaluate(val_input_fun)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'logs/func/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into logs/func/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 513.51624, step = 0
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 100...
INFO:tensorflow:Saving checkpoints for 100 into logs/func/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 100...
INFO:tensorflow:Loss for final step: 55.55261.
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-06-11T22:25:07Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from logs/func/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.26729s
INFO:tensorflow:Finished evaluation at 2021-06-11-22:25:08
INFO:tensorflow:Saving dict for global step 100: average_loss = 69.61237, global_step = 100, label/mean = 23.078432, loss = 82.44842, prediction/mean = 22.215874
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: logs/func/model.ckpt-100


In [12]:
# Make a prediction on it:
result = linear_est.predict(val_input_fun)
for pred,exp in zip(result, y_test[:32]):
    print("Predicted: ", pred['predictions'][0], "Expected:", exp)

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from logs/func/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predicted:  3.3225527 Expected: 7.2
Predicted:  22.66687 Expected: 18.8
Predicted:  21.299503 Expected: 19.0
Predicted:  22.66605 Expected: 27.0
Predicted:  21.883333 Expected: 22.2
Predicted:  21.10038 Expected: 24.5
Predicted:  29.275932 Expected: 31.2
Predicted:  25.054379 Expected: 22.9
Predicted:  19.889421 Expected: 20.5
Predicted:  22.97792 Expected: 23.2
Predicted:  8.811323 Expected: 18.6
Predicted:  22.16803 Expected: 14.5
Predicted:  22.366486 Expected: 17.8
Predicted:  22.248493 Expected: 50.0
Predicted:  17.720222 Expected: 20.8
Predicted:  24.292896 Expected: 24.3
Predicted:  20.047888 Expected: 24.2
Predicted:  20.77812 Expected: 19.8
Predicted:  22.76529 Expected: 19.1
Predicted:  25.10835 Expected: 22.7
Predicted:  18.519823 Expected: 12.0
Predict